### Prototype Code : Table Generator

In [ ]:
# Table File Generator

def dataframe_init(in_data, out_data, text_data):
    # initialize data of lists.
    data = {"output" : out_data, 
           "input" : in_data,
           "instruction" : text_data}

    # Create DataFrame
    df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in data.items()]))
    # df = pd.DataFrame(data)
    return df

In [ ]:
# CSV Generator

def csv_from_test_case_batches(filename, input_column, output_column, text_column):
    cols = dataframe_init(input_column, output_column, text_column)
    cols.to_csv(f"{filename}.csv", sep='\t', encoding='utf-8', index=False, header=True)

In [ ]:
# Main Program

# FILENAME
filename = "sample_name"
# URL
url = get_url()
# DATA
data = data_scrape(url) # Returns Link Data Only For Now

# Prepare data to be plcaed in the Input Column
input_data = []
for item_set in data:
    for item in item_set:
        input_data.append(item)

# CHAIN
chain = load_model_chain()
# GENERATE TEST CASES
test_cases = create_test_cases(data, chain)

# Prepare data to be plcaed in the Output Column [Split]
split_sets = []
for test_case_set in test_cases:
    for test_case in test_case_set:
        for test in test_case:
            split_sets.append(test.split('~'))
# Prepare data to be plcaed in the Output Column [Actual Column]
output_data = []
for sp in split_sets:
    output_data.append(sp)
    
# Fine-Tuning Template
text_data = []
i = 0
while (i < len(input_data)):
    template = f"####Question:\n Generate test case for the following UI element: {input_data[i]}"
    text_data.append(template)
    i+=1
    
# CSV OUTPUT
csv_from_test_case_batches(filename, input_data, output_data, text_data)

### Version 1 : Dataset Generator and System Proper Split

In [ ]:
# Table Dataframe Initialization

def dataframe_init(data):
    df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in data.items()]))
    return df

# CSV Generator
def csv_from_test_case_batches(filename, data):
    cols = dataframe_init(data)
    cols.to_csv(f"{filename}.csv", sep='\t', encoding='utf-8', index=False, header=True)

# Dataset Generator Parameters
out_data = None # LLM Output
in_data = None # Scraped UI Element
instruction_data = None # Prompt
data = {"output" : out_data, 
        "input" : in_data, 
        "instruction" : instruction_data}
dataset_generator_df = dataframe_init(data)

# System Proper Parameters
id = None
precondition = None
test_steps = None
expected_result = None 
actual_result = "Pass/Fail"
data = {"Test Case ID" : id,
        "Precondition" : precondition,
        "Test Steps" : test_steps,
        "Expected Result" : expected_result,
        "Actual Result" : actual_result}
system_proper_df = dataframe_init(data)

In [ ]:
# TODO : Process of organizing the LLM output [streamline process of unpacking the list output from llm into something that is easy to list down]
# LLM Output Organization

def create_fine_tuning_dataset(scraped_data, llm_output):
    # Prepare data to be placed in the Input Column
    """
    Structure of scraped_data:
        scraped_data
            item_set (e.g. buttons)
                item (e.g button)
    """
    input_data = []
    for item_set in scraped_data:
        for item in item_set:
            input_data.append(item)

    # Prepare data to be plcaed in the Output Column
    """
    Structure of llm_output:
        llm_output:
            output_set (e.g. buttons)
                output_item (test_case_parameters, split by ~)
    """
    split_sets = []
    for test_case_set in llm_output:
        for test_case in test_case_set:
            for test in test_case:
                split_sets.append(test.split('~'))
    # Prepare data to be plcaed in the Output Column [Actual Column]
    output_data = []
    for sp in split_sets:
        output_data.append(sp)
        
    # Fine-Tuning Template
    text_data = []
    i = 0
    while (i < len(input_data)):
        template = f"####Question:\n Generate test case for the following UI element: {input_data[i]}"
        text_data.append(template)
        i+=1

def create_table_dataset(scraped_data, llm_output):
    # Prepare data to be placed in the Input Column
    """
    Structure of scraped_data:
        scraped_data
            item_set (e.g. buttons)
                item (e.g button)
    """
    input_data = []
    for item_set in scraped_data:
        for item in item_set:
            input_data.append(item)

    # Prepare data to be plcaed in the Output Column
    """
    Structure of llm_output:
        llm_output:
            output_set (e.g. buttons)
                output_item (test_case_parameters, split by ~)
    """
    split_sets = []
    for test_case_set in llm_output:
        for test_case in test_case_set:
            for test in test_case:
                split_sets.append(test.split('~'))
    # Prepare data to be plcaed in the Output Column [Actual Column]
    output_data = []
    for sp in split_sets:
        # i = 0
        for s in sp:
            # i += 1
            # output_data[i].append(s)
            output_data.append(sp)
        
    # Fine-Tuning Template
    text_data = []
    i = 0
    while (i < len(input_data)):
        template = f"####Question:\n Generate test case for the following UI element: {input_data[i]}"
        text_data.append(template)
        i+=1